# 📘 Kỹ Thuật Imputation Trên Bộ Dữ Liệu Titanic
Notebook này trình bày toàn bộ các bước xử lý giá trị thiếu (missing values) bằng tiếng Việt.

## 📦 1. Import thư viện cần thiết

In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

## 🚢 2. Tải bộ dữ liệu Titanic

In [3]:
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = pd.read_csv(url)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 🔍 3. Kiểm tra giá trị thiếu

In [ ]:
df.isna().sum()

### 📊 Biểu đồ giá trị thiếu theo cột

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x=df.isna().sum().index, y=df.isna().sum().values)
plt.xticks(rotation=45)
plt.title('Số lượng giá trị thiếu trên từng cột')
plt.show()

## 🧩 4. Imputation đơn biến (Univariate)

In [ ]:
mean_imp = SimpleImputer(strategy='mean')
df['Age_mean'] = mean_imp.fit_transform(df[['Age']])

median_imp = SimpleImputer(strategy='median')
df['Fare_median'] = median_imp.fit_transform(df[['Fare']])

mode_imp = SimpleImputer(strategy='most_frequent')
df['Embarked_mode'] = mode_imp.fit_transform(df[['Embarked']]).ravel()

const_imp = SimpleImputer(strategy='constant', fill_value='Unknown')
df['Cabin_constant'] = const_imp.fit_transform(df[['Cabin']]).ravel()

df.head()

### 📈 So sánh phân phối Age trước và sau Mean Imputation

In [ ]:
plt.figure(figsize=(10,5))
sns.kdeplot(df['Age'], label='Age gốc', fill=True)
sns.kdeplot(df['Age_mean'], label='Age sau Mean Imputation', fill=True)
plt.legend()
plt.title('So sánh phân phối Age trước và sau Imputation')
plt.show()

In [ ]:
# Encode Sex
df['Sex_encoded'] = df['Sex'].map({'male': 0, 'female': 1})

# One-hot encode Embarked
embarked_ohe = pd.get_dummies(df['Embarked'], prefix='Embarked', dummy_na=True)

# Gộp vào dataframe
df = pd.concat([df, embarked_ohe], axis=1)

df.head()

## 🤖 5. Imputation đa biến — KNN (phiên bản chuẩn dùng nhiều biến)

In [ ]:

# Danh sách cột dùng cho multivariate
selected_cols = ['Age','Fare','Pclass','Sex_encoded','SibSp','Parch'] + list(embarked_ohe.columns)

df_selected = df[selected_cols]

# Áp dụng KNN nâng cao
knn_full = KNNImputer(n_neighbors=5)
df_knn_full = knn_full.fit_transform(df_selected)

df_knn_df = pd.DataFrame(df_knn_full, columns=[col + "_knn" for col in df_selected.columns])
df_knn_df.head()


## 🔁 6. Imputation đa biến — MICE (phiên bản chuẩn dùng nhiều biến)

In [ ]:
mice = IterativeImputer(max_iter=20, random_state=0)
df_mice_full = mice.fit_transform(df_selected)

df_mice_df = pd.DataFrame(df_mice_full, columns=[c + "_mice" for c in df_selected.columns])
df_mice_df.head()


## 🧷 7. Gộp toàn bộ kết quả

In [ ]:
final_df = pd.concat([df, df_knn_df, df_mice_df], axis=1)
final_df.head()

## ✅ 8. Kết luận
- Mean/Median/Mode phù hợp các trường hợp đơn giản.
- KNN giúp tận dụng cấu trúc dữ liệu.
- MICE tái tạo phân phối dữ liệu tốt nhất nhưng chậm hơn.
- Titanic Dataset rất phù hợp minh hoạ các kỹ thuật imputation.

## 📊 9. So sánh trực quan các phương pháp Imputation

### 🔍 So sánh phân phối Age: Mean vs KNN vs MICE

In [ ]:
plt.figure(figsize=(10,6))
sns.kdeplot(df["Age"], label="Age gốc", fill=True)
sns.kdeplot(df["Age_mean"], label="Mean Imputation", fill=True)
sns.kdeplot(df_knn_df["Age_knn"], label="KNN Imputation", fill=True)
sns.kdeplot(df_mice_df["Age_mice"], label="MICE Imputation", fill=True)
plt.title("So sánh phân phối Age của các phương pháp Imputation")
plt.legend()
plt.show()